# Interactive Image Annotation Tool (Standalone)

This notebook provides an interactive interface for annotating cricket images with 8×8 grid cells.

**This is a standalone version that doesn't require ml_utils modules.**

**Usage:**
1. Set the image path and annotation directory below
2. Run all cells
3. Use the dropdown menus to select cells and label them
4. Save your annotations when done

**Features:**
- Visual grid overlay with cell numbers (c01-c64)
- Click/select cells to label
- Color-coded highlights for different object types
- Easy save and load functionality
- Image browser with filtering and navigation


In [1]:
# Standard library imports
import numpy as np
from pathlib import Path
from typing import List, Optional
import re

# Third-party imports
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, clear_output
import ipywidgets as widgets

# Check ipywidgets installation
try:
    print(f"ipywidgets version: {widgets.__version__}")
except:
    print("Warning: Could not get ipywidgets version")


ipywidgets version: 8.1.5


## Configuration Constants

Set up the grid configuration and class mappings.


In [2]:
# Image processing constants
TARGET_WIDTH = 800
TARGET_HEIGHT = 600

# Grid configuration
GRID_SIZE = 8
CELL_WIDTH = TARGET_WIDTH // GRID_SIZE  # 100 pixels
CELL_HEIGHT = TARGET_HEIGHT // GRID_SIZE  # 75 pixels

# Class mapping
CLASS_NAMES = {0: "no_object", 1: "ball", 2: "bat", 3: "stump"}


## Utility Functions

Functions for loading/saving annotations and creating visualizations.


In [3]:
def save_grid_annotation(grid_labels: List[int], output_path: Path) -> None:
    """
    Save grid cell labels to file.

    Args:
        grid_labels: List of 64 cell labels (0-3)
        output_path: Path to save the annotation
    """
    output_path.parent.mkdir(parents=True, exist_ok=True)

    with open(output_path, 'w') as f:
        for i, label in enumerate(grid_labels):
            f.write(f"c{i+1:02d},{label}\n")


def load_grid_annotation(annotation_path: Path) -> Optional[List[int]]:
    """
    Load grid cell labels from file.

    Args:
        annotation_path: Path to saved grid annotation (c01,<label>) lines

    Returns:
        List of 64 integers if file exists and valid, otherwise None
    """
    if not annotation_path.exists():
        return None
    labels: List[int] = []
    with open(annotation_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(',')
            if len(parts) != 2:
                continue
            try:
                labels.append(int(parts[1]))
            except ValueError:
                continue
    if len(labels) != 64:
        return None
    return labels


def _create_grid_visualization(img: Image.Image, labels: List[int], selected_cells: Optional[List[int]] = None) -> Image.Image:
    """
    Create visualization with grid overlay and cell numbers.

    Args:
        img: PIL Image object
        labels: List of 64 cell labels (0-3)
        selected_cells: Optional list of selected cell indices

    Returns:
        PIL Image with grid overlay
    """
    vis_img = img.copy()
    draw = ImageDraw.Draw(vis_img)

    # Try to load fonts
    try:
        font_small = ImageFont.truetype("Arial.ttf", 10)
        font_large = ImageFont.truetype("Arial.ttf", 12)
    except Exception:
        font_small = ImageFont.load_default()
        font_large = ImageFont.load_default()

    # Draw grid lines
    for i in range(GRID_SIZE + 1):
        x = i * CELL_WIDTH
        draw.line([(x, 0), (x, TARGET_HEIGHT)], fill="red", width=2)
        y = i * CELL_HEIGHT
        draw.line([(0, y), (TARGET_WIDTH, y)], fill="red", width=2)

    # Draw cells
    for row in range(GRID_SIZE):
        for col in range(GRID_SIZE):
            cell_idx = row * GRID_SIZE + col
            cell_num = cell_idx + 1
            label = labels[cell_idx]

            left = col * CELL_WIDTH
            top = row * CELL_HEIGHT
            center_x = left + CELL_WIDTH // 2
            center_y = top + CELL_HEIGHT // 2

            # Highlight selected cell
            if selected_cells and cell_idx in selected_cells:
                draw.rectangle(
                    [(left, top), (left + CELL_WIDTH, top + CELL_HEIGHT)],
                    outline="yellow",
                    width=3,
                )

            # Draw cell number
            draw.text(
                (left + 2, top + 2),
                f"c{cell_num:02d}",
                fill="white",
                font=font_small,
                stroke_width=1,
                stroke_fill="black",
            )

            # Highlight cells with objects
            if label != 0:
                overlay = Image.new("RGBA", (CELL_WIDTH, CELL_HEIGHT), (0, 0, 0, 0))
                overlay_draw = ImageDraw.Draw(overlay)
                colors = {
                    1: (255, 255, 0, 80),  # yellow for ball
                    2: (0, 255, 0, 80),   # green for bat
                    3: (0, 0, 255, 80),   # blue for stump
                }
                color = colors.get(label, (0, 0, 0, 0))
                overlay_draw.rectangle([(0, 0), (CELL_WIDTH, CELL_HEIGHT)], fill=color)
                vis_img.paste(overlay, (left, top), overlay)

                # Draw class label
                class_name = CLASS_NAMES[label]
                label_text = f"{label}:{class_name[:4]}"
                text_width = draw.textlength(label_text, font=font_large)
                draw.text(
                    (center_x - text_width // 2, center_y - 6),
                    label_text,
                    fill="white",
                    font=font_large,
                    stroke_width=2,
                    stroke_fill="black",
                )

    return vis_img


## Annotation Widget Class

Interactive widget for annotating cells in an image.


In [4]:
class AnnotationWidget:
    def __init__(self, image_path: Path, annotations_dir: Path, initial_labels: Optional[List[int]] = None) -> None:
        self.image_path = image_path
        self.annotations_dir = annotations_dir
        self.labels = (initial_labels or [0] * 64).copy()
        self.selected_cells: List[int] = []
        self.output = widgets.Output()

        self.class_selector = widgets.Dropdown(
            options=[(CLASS_NAMES[i], i) for i in range(4)],
            value=2,
            description="Class:",
            style={"description_width": "initial"},
        )
        self.cell_selector = widgets.SelectMultiple(
            options=[(f"c{i+1:02d} (row {i//8}, col {i%8})", i) for i in range(64)],
            value=[],
            description="Select Cells:",
            style={"description_width": "initial"},
        )
        self.set_button = widgets.Button(description="Set Label", button_style="success")
        self.clear_button = widgets.Button(description="Clear Cells", button_style="warning")
        self.save_button = widgets.Button(description="Save Annotation", button_style="primary")
        self.refresh_button = widgets.Button(description="Refresh Preview")
        self.status = widgets.HTML()

        self.set_button.on_click(self._on_set_click)
        self.clear_button.on_click(self._on_clear_click)
        self.save_button.on_click(self._on_save_click)
        self.refresh_button.on_click(self._on_refresh_click)
        self.cell_selector.observe(self._on_cell_change, names="value")

        self._update_display()

    def set_image(self, image_path: Path, labels: Optional[List[int]] = None) -> None:
        self.image_path = image_path
        if labels is not None:
            self.labels = labels.copy()
        self.selected_cells = []
        self.cell_selector.value = []
        self._update_display()

    def _on_set_click(self, _b) -> None:
        selected_indices = list(self.cell_selector.value)
        if not selected_indices:
            self.status.value = "<b style='color:orange'>⚠ Please select at least one cell</b>"
            return

        class_id = self.class_selector.value
        changed_cells = []
        for cell_idx in selected_indices:
            old_label = self.labels[cell_idx]
            self.labels[cell_idx] = class_id
            changed_cells.append((cell_idx, old_label))

        self._update_display()

        if len(changed_cells) == 1:
            cell_idx, old_label = changed_cells[0]
            self.status.value = (
                f"<b>Cell c{cell_idx+1:02d}</b> set to <b>{CLASS_NAMES[class_id]}</b> (was {CLASS_NAMES[old_label]})"
            )
        else:
            cell_names = ", ".join([f"c{idx+1:02d}" for idx in selected_indices])
            self.status.value = (
                f"<b>{len(selected_indices)} cells</b> ({cell_names}) set to <b>{CLASS_NAMES[class_id]}</b>"
            )

    def _on_clear_click(self, _b) -> None:
        selected_indices = list(self.cell_selector.value)
        if not selected_indices:
            self.status.value = "<b style='color:orange'>⚠ Please select at least one cell</b>"
            return

        for cell_idx in selected_indices:
            self.labels[cell_idx] = 0

        self._update_display()

        if len(selected_indices) == 1:
            self.status.value = f"<b>Cell c{selected_indices[0]+1:02d}</b> cleared"
        else:
            cell_names = ", ".join([f"c{idx+1:02d}" for idx in selected_indices])
            self.status.value = f"<b>{len(selected_indices)} cells</b> ({cell_names}) cleared"

    def _on_save_click(self, _b) -> None:
        ann_path = self.annotations_dir / f"{self.image_path.stem}.txt"
        save_grid_annotation(self.labels, ann_path)
        class_counts = {i: sum(1 for x in self.labels if x == i) for i in range(4)}
        counts_str = ", ".join(
            [f"{CLASS_NAMES[i]}: {class_counts[i]}" for i in range(4) if class_counts[i] > 0]
        )
        self.status.value = f"<b style='color:green'>✓ Saved!</b> {ann_path.name}<br>Summary: {counts_str}"

    def _on_refresh_click(self, _b) -> None:
        self._update_display()

    def _on_cell_change(self, change) -> None:
        # SelectMultiple returns a tuple, convert to list
        self.selected_cells = list(change["new"]) if change["new"] else []
        self._update_display()

    def _update_display(self) -> None:
        with self.output:
            clear_output(wait=True)
            img = Image.open(self.image_path)
            vis_img = _create_grid_visualization(img, self.labels, self.selected_cells)
            display(vis_img)

    def get_widget(self) -> widgets.Widget:
        controls = widgets.VBox(
            [
                widgets.HTML("<h3>📝 Annotation Controls</h3>"),
                self.cell_selector,
                self.class_selector,
                widgets.HBox([self.set_button, self.clear_button]),
                widgets.HBox([self.save_button, self.refresh_button]),
                self.status,
            ]
        )
        return widgets.VBox([self.output, controls])


## Image Browser Widget Class

Widget for browsing, filtering, and navigating through images.


In [5]:
class ImageBrowserWidget:
    def __init__(self, images_dir: Path, annotations_dir: Path, annotation_widget: AnnotationWidget) -> None:
        self.images_dir = images_dir
        self.annotations_dir = annotations_dir
        self.annotation_widget = annotation_widget

        self.dir_text = widgets.Text(
            value=str(images_dir), description="Images dir:", layout=widgets.Layout(width="70%")
        )
        self.refresh_btn = widgets.Button(description="Refresh")
        self.filter_text = widgets.Text(value="", description="Filter:")
        self.unannotated_only = widgets.Checkbox(value=False, description="Only unannotated")

        self.image_dropdown = widgets.Dropdown(options=[], description="Image:", layout=widgets.Layout(width="70%"))
        self.first_btn = widgets.Button(description="⏮ First")
        self.prev_btn = widgets.Button(description="◀ Prev")
        self.next_btn = widgets.Button(description="Next ▶")
        self.last_btn = widgets.Button(description="Last ⏭")
        self.skip_btn = widgets.Button(description="Skip")
        self.delete_btn = widgets.Button(description="Delete", button_style="danger")
        self.rename_text = widgets.Text(
            value="",
            description="Rename to:",
            placeholder="new_name.jpg",
            layout=widgets.Layout(width="50%"),
        )
        self.rename_btn = widgets.Button(description="Rename", button_style="info")
        self.progress_html = widgets.HTML()
        self.status_html = widgets.HTML()

        self.browser_images: List[Path] = []
        self.annotated_set: set[str] = set()
        self.current_index = -1

        self._suspend_dropdown_events: bool = False

        self.dir_text.observe(self._on_dir_changed, names="value")
        self.refresh_btn.on_click(self._on_refresh)
        self.filter_text.observe(self._on_filter_changed, names="value")
        self.unannotated_only.observe(self._on_filter_changed, names="value")
        self.image_dropdown.observe(self._on_dropdown_changed, names="value")
        self.first_btn.on_click(lambda _b: self._select_index(0))
        self.prev_btn.on_click(lambda _b: self._select_index(max(0, self.current_index - 1)))
        self.next_btn.on_click(lambda _b: self._select_index(min(len(self.browser_images) - 1, self.current_index + 1)))
        self.last_btn.on_click(lambda _b: self._select_index(len(self.browser_images) - 1))
        self.skip_btn.on_click(self._on_skip)
        self.delete_btn.on_click(self._on_delete)
        self.rename_btn.on_click(self._on_rename)

        self._scan_images()

    def get_widget(self) -> widgets.Widget:
        ui = widgets.VBox(
            [
                widgets.HBox([self.dir_text, self.refresh_btn]),
                widgets.HBox([self.filter_text, self.unannotated_only]),
                widgets.HBox([self.image_dropdown]),
                widgets.HBox([self.first_btn, self.prev_btn, self.next_btn, self.last_btn, self.skip_btn, self.delete_btn]),
                widgets.HBox([self.rename_text, self.rename_btn]),
                self.progress_html,
                self.status_html,
            ]
        )
        return ui

    def _scan_images(self) -> None:
        base = Path(self.dir_text.value).expanduser().resolve()
        imgs = sorted(list(base.glob("*.jpg")))
        annotated = set()
        for p in imgs:
            if (self.annotations_dir / f"{p.stem}.txt").exists():
                annotated.add(p.name)
        self.annotated_set = annotated

        q = self.filter_text.value.strip()
        only_un = self.unannotated_only.value
        filtered: List[Path] = []
        for p in imgs:
            name = p.name
            if q:
                try:
                    if re.search(q, name) is None:
                        continue
                except Exception:
                    if q.lower() not in name.lower():
                        continue
            if only_un and (name in annotated):
                continue
            filtered.append(p)

        self.browser_images = filtered
        self.image_dropdown.options = [(p.name, i) for i, p in enumerate(self.browser_images)] or [("— no images —", None)]
        self.current_index = 0 if self.browser_images else -1
        if self.current_index >= 0:
            self.image_dropdown.value = 0
        self._update_progress()

    def _update_progress(self) -> None:
        total = len(self.browser_images)
        ann = sum(1 for p in self.browser_images if p.name in self.annotated_set)
        un = total - ann
        idx_disp = (self.current_index + 1) if self.current_index >= 0 else 0
        self.progress_html.value = f"<b>Progress:</b> {idx_disp}/{total} • Annotated: {ann} • Unannotated: {un}"

    def _select_index(self, i: int) -> None:
        if not self.browser_images:
            return
        i = max(0, min(i, len(self.browser_images) - 1))
        self.current_index = i
        self.image_dropdown.value = i
        self._apply_selection()

    def _apply_selection(self) -> None:
        if self.current_index < 0 or not self.browser_images:
            return
        image_path = self.browser_images[self.current_index]
        ann_path = self.annotations_dir / f"{image_path.stem}.txt"
        labels = load_grid_annotation(ann_path) or [0] * 64
        self.annotation_widget.set_image(image_path, labels)
        self._update_progress()

    # Event handlers
    def _on_refresh(self, _b) -> None:
        self._scan_images()

    def _on_dir_changed(self, change) -> None:
        if change["name"] == "value":
            self._scan_images()

    def _on_filter_changed(self, change) -> None:
        if change["name"] == "value":
            self._scan_images()

    def _on_dropdown_changed(self, change) -> None:
        if self._suspend_dropdown_events:
            return
        if change["name"] == "value" and change["new"] is not None and self.browser_images:
            self._select_index(change["new"])

    def _on_skip(self, _b) -> None:
        if not self.browser_images or self.current_index < 0:
            return
        p = self.browser_images.pop(self.current_index)
        self.browser_images.append(p)
        self.image_dropdown.options = [(pp.name, i) for i, pp in enumerate(self.browser_images)]
        self._select_index(min(self.current_index, len(self.browser_images) - 1))

    def _on_delete(self, _b) -> None:
        if not self.browser_images or self.current_index < 0:
            self.status_html.value = "<span style='color:red'>No image to delete.</span>"
            return
        p = self.browser_images[self.current_index]
        ann_path = self.annotations_dir / f"{p.stem}.txt"
        errs: List[str] = []
        try:
            if p.exists():
                p.unlink()
        except Exception as e:
            errs.append(str(e))
        try:
            if ann_path.exists():
                ann_path.unlink()
        except Exception as e:
            errs.append(str(e))
        if p.name in self.annotated_set:
            self.annotated_set.discard(p.name)
        deleted_index = self.current_index
        self.browser_images.pop(deleted_index)

        # If list becomes empty
        if not self.browser_images:
            self.status_html.value = "<span style='color:orange'>Deleted. No images left.</span>"
            return

        # Decide next index: if deleted last item, go to first (index 0), else stay at same index (now next item)
        if deleted_index >= len(self.browser_images):
            next_index = 0
        else:
            next_index = deleted_index

        # Update dropdown without firing change handler
        self._suspend_dropdown_events = True
        try:
            self.image_dropdown.options = [(pp.name, i) for i, pp in enumerate(self.browser_images)]
            self.current_index = next_index
            self.image_dropdown.value = next_index
        finally:
            self._suspend_dropdown_events = False

        # Apply selection to load the correct image and update progress
        self._apply_selection()
        self.status_html.value = (
            "<span style='color:green'>Deleted image and annotation (if existed).</span>"
            if not errs
            else f"<span style='color:orange'>Deleted with warnings: {'; '.join(errs)}</span>"
        )

    def _on_rename(self, _b) -> None:
        if not self.browser_images or self.current_index < 0:
            self.status_html.value = "<span style='color:red'>No image to rename.</span>"
            return
        new_name = self.rename_text.value.strip()
        if not new_name:
            self.status_html.value = "<span style='color:red'>Enter a new name.</span>"
            return
        if not new_name.lower().endswith(".jpg"):
            new_name = f"{Path(new_name).stem}.jpg"
        p = self.browser_images[self.current_index]
        new_path = p.with_name(new_name)
        if new_path.exists():
            self.status_html.value = "<span style='color:red'>File already exists.</span>"
            return
        # Step 1: rename the image file
        try:
            p.rename(new_path)
        except Exception as e:
            self.status_html.value = f"<span style='color:red'>Rename failed: {e}</span>"
            return

        # Step 2: attempt to rename the annotation; on failure, rollback image rename
        old_ann = self.annotations_dir / f"{p.stem}.txt"
        new_ann = self.annotations_dir / f"{new_path.stem}.txt"
        try:
            if old_ann.exists():
                old_ann.rename(new_ann)
        except Exception as e:
            # Try to rollback the image rename
            try:
                new_path.rename(p)
                self.status_html.value = (
                    f"<span style='color:orange'>Annotation rename failed and image rename was reverted. Reason: {e}</span>"
                )
                return
            except Exception as e2:
                # Rollback failed; leave UI consistent with filesystem (image at new_path)
                self.status_html.value = (
                    f"<span style='color:red'>Annotation rename failed ({e}) and rollback failed ({e2}). Files may be inconsistent.</span>"
                )
                # Proceed to update internal lists to reflect the on-disk state (new_path)

        # Step 3: update internal state reflecting successful rename
        if p.name in self.annotated_set:
            self.annotated_set.discard(p.name)
            # Only add if new annotation exists (after successful rename)
            if new_ann.exists():
                self.annotated_set.add(new_path.name)
        self.browser_images[self.current_index] = new_path
        # Update dropdown without triggering a jump to index 0
        self._suspend_dropdown_events = True
        try:
            self.image_dropdown.options = [(pp.name, i) for i, pp in enumerate(self.browser_images)]
            self.image_dropdown.value = self.current_index
        finally:
            self._suspend_dropdown_events = False
        self._apply_selection()
        self.status_html.value = f"<span style='color:green'>Renamed to {new_path.name}.</span>"


## Build Annotation App Function

Function to create the complete annotation application.


In [6]:
def build_annotation_app(images_dir: Path, annotations_dir: Path) -> widgets.Widget:
    """High-level helper to build a single-pane app with browser + annotation."""
    first_image = next((p for p in sorted(Path(images_dir).glob("*.jpg"))), None)
    if first_image is None:
        blank = Image.new("RGB", (TARGET_WIDTH, TARGET_HEIGHT), (255, 255, 255))
        tmp = annotations_dir / "__blank__.jpg"
        tmp.parent.mkdir(parents=True, exist_ok=True)
        blank.save(tmp, format="JPEG")
        first_image = tmp

    labels = load_grid_annotation(annotations_dir / f"{first_image.stem}.txt") or [0] * 64
    annot = AnnotationWidget(first_image, annotations_dir, labels)
    browser = ImageBrowserWidget(images_dir, annotations_dir, annot)
    return widgets.VBox([browser.get_widget(), annot.get_widget()])


## Usage

Configure your image and annotation directories below, then run the cell to start annotating.


In [ ]:
# Configure your directories here
# Update these paths to point to your image and annotation directories

# Example paths (adjust as needed):
# IMAGES_DIR = Path("data/processed/atul")
# ANNOTATIONS_DIR = Path("data/annotations/annotations/atul")

# Or use absolute paths:
IMAGES_DIR = Path("C:\\Users\\pravi\\PG IITB\\cricket_object_detection\\data\\train\\bat")
ANNOTATIONS_DIR = Path("C:\\Users\\pravi\\PG IITB\\cricket_object_detection\\data\\annotations\\pravintp")

# Create annotation directory if it doesn't exist
ANNOTATIONS_DIR.mkdir(parents=True, exist_ok=True)

# Build and display the annotation app
app = build_annotation_app(IMAGES_DIR, ANNOTATIONS_DIR)
display(app)
